In [3]:
from music21 import *
import tensorflow
import sys
import os
import IPython
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import random

2023-04-10 21:46:36.176843: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 21:46:36.605191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-10 21:46:36.605219: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-10 21:46:39.500574: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
environment.set('musescoreDirectPNGPath', '../.local/bin/MuseScore-4.0.1.230121751-x86_64.AppImage')

In [6]:
filepath = "dataset/"

In [7]:
all_midis= []
names = []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath + i
        names.append(tr)
        midi = converter.parse(tr)
        all_midis.append(midi)

        

dataset/737.mid
dataset/748.mid
dataset/754.mid
dataset/573.mid
dataset/374.mid
dataset/488.mid
dataset/151.mid
dataset/850.mid
dataset/122.mid
dataset/634.mid
dataset/290.mid
dataset/404.mid
dataset/773.mid
dataset/624.mid
dataset/442.mid
dataset/698.mid
dataset/470.mid
dataset/420.mid
dataset/604.mid
dataset/455.mid
dataset/887.mid
dataset/255.mid
dataset/087.mid
dataset/452.mid
dataset/644.mid
dataset/215.mid
dataset/702.mid
dataset/728.mid
dataset/168.mid
dataset/341.mid
dataset/877.mid
dataset/164.mid
dataset/197.mid
dataset/595.mid
dataset/904.mid
dataset/416.mid
dataset/017.mid
dataset/229.mid
dataset/509.mid
dataset/303.mid
dataset/521.mid
dataset/519.mid
dataset/591.mid
dataset/169.mid
dataset/852.mid
dataset/720.mid
dataset/636.mid
dataset/298.mid
dataset/869.mid
dataset/398.mid
dataset/115.mid
dataset/265.mid
dataset/701.mid
dataset/781.mid
dataset/811.mid
dataset/523.mid
dataset/667.mid
dataset/225.mid
dataset/763.mid
dataset/829.mid
dataset/535.mid
dataset/524.mid
dataset/

dataset/013.mid
dataset/314.mid
dataset/633.mid
dataset/010.mid
dataset/240.mid
dataset/632.mid
dataset/132.mid
dataset/430.mid
dataset/553.mid
dataset/267.mid
dataset/579.mid
dataset/518.mid
dataset/338.mid
dataset/254.mid
dataset/352.mid
dataset/432.mid
dataset/206.mid
dataset/009.mid
dataset/870.mid
dataset/750.mid
dataset/411.mid
dataset/084.mid
dataset/680.mid
dataset/357.mid
dataset/434.mid
dataset/142.mid
dataset/897.mid
dataset/361.mid
dataset/474.mid
dataset/353.mid
dataset/463.mid
dataset/116.mid
dataset/560.mid
dataset/740.mid
dataset/242.mid
dataset/706.mid
dataset/421.mid
dataset/550.mid
dataset/599.mid
dataset/335.mid
dataset/854.mid
dataset/003.mid
dataset/707.mid
dataset/534.mid
dataset/607.mid
dataset/661.mid
dataset/256.mid
dataset/853.mid
dataset/699.mid
dataset/584.mid
dataset/655.mid
dataset/886.mid
dataset/665.mid
dataset/627.mid
dataset/522.mid
dataset/083.mid
dataset/659.mid
dataset/238.mid
dataset/499.mid
dataset/163.mid
dataset/505.mid
dataset/032.mid
dataset/

In [8]:
def extract_notes(files):
    notes = []
    pick = None
    for f in files:
        #songs = instrument.partitionByInstrument(f)
        for part in f.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))

    return notes
#Getting the list of notes as Corpus
Corpus= np.array(extract_notes(all_midis))
#print("Total notes in all the Mozzart midis in the dataset:", len(Corpus))

In [10]:
Corpus = np.reshape(Corpus, (len(Corpus), 1))

In [14]:
# Initialize OneHotEncoder object
encoder = OneHotEncoder()

# Fit and transform the data
one_hot_data = encoder.fit_transform(Corpus)

# Convert sparse matrix to array
one_hot_data = one_hot_data.toarray()

# Print the one-hot encoded data
print(one_hot_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
input_len = 16

In [17]:
def getXy(data, step):
    y_ind = np.arange(step, len(data), step)
    y = data[y_ind]
    rows_x = len(y)
    X = data[range(rows_x * step)]
    X = np.reshape(X, (rows_x, step, one_hot_data.shape[1]))
    return X, y

In [18]:
print(len(one_hot_data))
indeksi = np.arange(input_len, len(one_hot_data),input_len)
one_hot_data[indeksi].shape

332243


(20765, 55)

In [19]:
X, y = getXy(one_hot_data, input_len)

In [23]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(LSTM(hidden_units, input_shape=input_shape))
    model.add(Dropout(0.25))
    model.add(Dense(units=dense_units))
    model.add(Dense(units=one_hot_data.shape[1], activation='softmax'))
    model.compile(loss=tensorflow.losses.CategoricalCrossentropy(), optimizer='adam')
    return model

In [24]:
model = create_RNN(1024, 328, input_shape=(input_len, one_hot_data.shape[1]), activation=['softmax', 'softmax'])

2023-04-10 21:49:13.367485: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-10 21:49:13.367847: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-10 21:49:13.367907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (milos): /proc/driver/nvidia/version does not exist
2023-04-10 21:49:13.368909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 21:49:13.492184: W tensorflow/tsl/framework/cpu_allocator_impl.cc:8

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1024)              4423680   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 328)               336200    
                                                                 
 dense_1 (Dense)             (None, 55)                18095     
                                                                 
Total params: 4,777,975
Trainable params: 4,777,975
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=25)

Epoch 1/25
649/649 [==============================] - 326s 498ms/step - loss: 2.3508
Epoch 2/25
649/649 [==============================] - 316s 487ms/step - loss: 2.0910
Epoch 3/25
649/649 [==============================] - 311s 480ms/step - loss: 2.0360
Epoch 4/25
448/649 [===================>..........] - ETA: 1:44 - loss: 2.0005

In [ ]:
model.save('mg_model.h5')

In [47]:
#currently not used
scales_with_5th = {
  "c-major": ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5'],
  "c#-major" : ['A#4', 'C4', 'C#4', 'D#4', 'F4', 'F#4', 'G#4', 'A#5', 'C5', 'C#5', 'D#5', 'F5', 'F#5', 'G#5'],
  "d-major": ['B4', 'C#4', 'D4', 'E4', 'F#4', 'G4', 'A4', 'B5', 'C#5', 'D5', 'E5', 'F#5', 'G5', 'A5'],
  "d#-major": ['C4', 'D4', 'D#4', 'F4', 'G4', 'G#4', 'A#4', 'C5', 'D5', 'D#5', 'F5', 'G5', 'G#5', 'A#5'],
  "e-major": ['C#4', 'D#4', 'E4', 'F#4', 'G#4', 'A4', 'B4', 'C#5', 'D#5', 'E5', 'F#5', 'G#5', 'A5', 'B5'],
  "f-major": ['D4', 'E4', 'F4', 'G4', 'A4', 'A#4', 'C4','D5', 'E5', 'F5', 'G5', 'A5', 'A#5', 'C5'],
  "f#-major": ['D#4', 'F4', 'F#4', 'G#4', 'A#4', 'B4', 'C#4', 'D#5', 'F5', 'F#5', 'G#5', 'A#5', 'B5', 'C#5'],
  "g-major": ['E4', 'F#4', 'G4', 'A4', 'B4', 'C4', 'D4', 'E5', 'F#5', 'G5', 'A5', 'B5', 'C5', 'D5'],
  "g#-major": ['F4', 'G4', 'G#4', 'A#4', 'C4', 'C#4', 'D#4', 'F5', 'G5', 'G#5', 'A#5', 'C5', 'C#5', 'D#5'],
  "a-major": ['F#4', 'G#4', 'A4', 'B4', 'C#4', 'D4', 'E4', 'F#5', 'G#5', 'A5', 'B5', 'C#5', 'D5', 'E5'],
  "a#-major": ['G4', 'A4', 'A#4', 'C4', 'D4', 'D#4', 'F4', 'G5', 'A5', 'A#5', 'C5', 'D5', 'D#5', 'F5'],
  "b-major": ['G#4', 'A#4', 'B4', 'C#4', 'D#4', 'E4', 'F#4', 'G#5', 'A#5', 'B5', 'C#5', 'D#5', 'E5', 'F#5'],
  
  "a-minor": ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5'],
  "a#-minor" : ['A#4', 'C4', 'C#4', 'D#4', 'F4', 'F#4', 'G#4', 'A#5', 'C5', 'C#5', 'D#5', 'F5', 'F#5', 'G#5'],
  "b-minor": ['B4', 'C#4', 'D4', 'E4', 'F#4', 'G4', 'A4', 'B5', 'C#5', 'D5', 'E5', 'F#5', 'G5', 'A5'],
  "c-minor": ['C4', 'D4', 'D#4', 'F4', 'G4', 'G#4', 'A#4', 'C5', 'D5', 'D#5', 'F5', 'G5', 'G#5', 'A#5'],
  "c#-minor": ['C#4', 'D#4', 'E4', 'F#4', 'G#4', 'A4', 'B4', 'C#5', 'D#5', 'E5', 'F#5', 'G#5', 'A5', 'B5'],
  "d-minor": ['D4', 'E4', 'F4', 'G4', 'A4', 'A#4', 'C4','D5', 'E5', 'F5', 'G5', 'A5', 'A#5', 'C5', '5.9.0','9.0.4'],
  "d#-minor": ['D#4', 'F4', 'F#4', 'G#4', 'A#4', 'B4', 'C#4', 'D#5', 'F5', 'F#5', 'G#5', 'A#5', 'B5', 'C#5'],
  "e-minor": ['E4', 'F#4', 'G4', 'A4', 'B4', 'C4', 'D4', 'E5', 'F#5', 'G5', 'A5', 'B5', 'C5', 'D5'],
  "f-minor": ['F4', 'G4', 'G#4', 'A#4', 'C4', 'C#4', 'D#4', 'F5', 'G5', 'G#5', 'A#5', 'C5', 'C#5', 'D#5'],
  "f#-minor": ['F#4', 'G#4', 'A4', 'B4', 'C#4', 'D4', 'E4', 'F#5', 'G#5', 'A5', 'B5', 'C#5', 'D5', 'E5'],
  "g-minor": ['G4', 'A4', 'A#4', 'C4', 'D4', 'D#4', 'F4', 'G5', 'A5', 'A#5', 'C5', 'D5', 'D#5', 'F5'],
  "g#-minor": ['G#4', 'A#4', 'B4', 'C#4', 'D#4', 'E4', 'F#4', 'G#5', 'A#5', 'B5', 'C#5', 'D#5', 'E5', 'F#5'],
}


In [48]:
scales = {
  "c-major": ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4'],
  "c#-major" : ['B-4', 'C4', 'C#4', 'E-4', 'F4', 'F#4', 'G#4'],
  "d-major": ['B4', 'C#4', 'D4', 'E4', 'F#4', 'G4', 'A4'],
  "e--major": ['C4', 'D4', 'E-4', 'F4', 'G4', 'G#4', 'B-4'],
  "e-major": ['C#4', 'E-4', 'E4', 'F#4', 'G#4', 'A4', 'B4'],
  "f-major": ['D4', 'E4', 'F4', 'G4', 'A4', 'B-4', 'C4'],
  "f#-major": ['E-4', 'F4', 'F#4', 'G#4', 'B-4', 'B4', 'C#4'],
  "g-major": ['E4', 'F#4', 'G4', 'A4', 'B4', 'C4', 'D4'],
  "g#-major": ['F4', 'G4', 'G#4', 'B-4', 'C4', 'C#4', 'E-4'],
  "a-major": ['F#4', 'G#4', 'A4', 'B4', 'C#4', 'D4', 'E4'],
  "b--major": ['G4', 'A4', 'B-4', 'C4', 'D4', 'E-4', 'F4'],
  "b-major": ['G#4', 'B-4', 'B4', 'C#4', 'E-4', 'E4', 'F#4'],
  
  "a-minor": ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4'],
  "b--minor" : ['B-4', 'C4', 'C#4', 'E-4', 'F4', 'F#4', 'G#4'],
  "b-minor": ['B4', 'C#4', 'D4', 'E4', 'F#4', 'G4', 'A4'],
  "c-minor": ['C4', 'D4', 'E-4', 'F4', 'G4', 'G#4', 'B-4'],
  "c#-minor": ['C#4', 'E-4', 'E4', 'F#4', 'G#4', 'A4', 'B4'],
  "d-minor": ['D4', 'E4', 'F4', 'G4', 'A4', 'B-4', 'C4'],
  "e--minor": ['E-4', 'F4', 'F#4', 'G#4', 'B-4', 'B4', 'C#4'],
  "e-minor": ['E4', 'F#4', 'G4', 'A4', 'B4', 'C4', 'D4'],
  "f-minor": ['F4', 'G4', 'G#4', 'B-4', 'C4', 'C#4', 'E-4'],
  "f#-minor": ['F#4', 'G#4', 'A4', 'B4', 'C#4', 'D4', 'E4'],
  "g-minor": ['G4', 'A4', 'B-4', 'C4', 'D4', 'E-4', 'F4'],
  "g#-minor": ['G#4', 'B-4', 'B4', 'C#4', 'E-4', 'E4', 'F#4']
}


In [31]:
input_scale = input('choose scale')

choose scalee-minor


In [99]:
#necemo random
#new_corpus = np.array([random.sample(scales[input_scale], input_len)])
indices = [0, 1, 2, 1, 6, 0, 2, 3, 2, 4, 3, 2, 1, 0, 6, 0]
new_corpus = []
for ind in indices:
    new_corpus.append(scales[input_scale][ind])
new_corpus = np.array([new_corpus])
#new_corpus = np.array(['E4', 'F#4', 'G4', 'F#4', 'D4', 'E4', 'G4', 'A4', 'G4', 'B4', 'A4', 'G4', 'F#4', 'E4', 'D4', 'E4'])
new_corpus = np.reshape(new_corpus, (1,-1))

In [101]:
new_corpus = np.reshape(new_corpus, (len(new_corpus[0]), 1))

In [103]:
#check occurences of notes in each scale

#for k, v in scales.items():
#    print(k)
#    print()
#    for n in v:
#        print(len(list(filter(lambda x : x == n, Corpus))))

In [44]:
all_notes = ['A4', 'B-4', 'B4', 'C4', 'C#4', 'D4', 'E-4','E4', 'F4','F#4', 'G4','G#4']

0


In [49]:
for n in all_notes:
    print(len(list(filter(lambda x : x == n, Corpus))))

7945


In [104]:
one_hot_new_data = encoder.transform(new_corpus)
one_hot_new_data = one_hot_new_data.toarray()

In [106]:
def funkcija(broj):
    if broj > 0.2:
        return 1
    else:
        return 0

In [107]:
acc_chords = {
    'c-major' : ['C3', 'E3', 'G3'],
    'c#-major' : ['C#3', 'F3', 'G#3'],
    'd-major' : ['D3', 'A3', 'F#3'],
    'e--major': ['E-3', 'B-3', 'G3'],
    'e-major' : ['E3', 'B3', 'G#3'],
    'f-major' : ['F3', 'C3', 'A3'],
    'f#-major' : ['F#3', 'C#3,' 'B-3'],
    'g-major' : ['G3', 'B3', 'D3'],
    'g#-major' : ['G#3', 'C3', 'E-3'],
    'a-major' : ['A3', 'E3', 'C#3'],
    'b--major': ['B-3', 'F3', 'D3'],
    'b-major' : ['B3', 'F#3', 'E-3'],
    
    'e-minor' : ['E3', 'B3', 'G3'],
    'f-minor' : ['F3', 'C3', 'G#3'],
    'f#-minor' : ['F#3', 'C#3', 'A3'],
    'g-minor' : ['G3', 'D3', 'B-3'],
    'g#-minor' : ['G#3', 'E-3', 'B3'],
    'a-minor' : ['A3', 'E3', 'C3'],
    'b--minor' : ['B-3', 'F3', 'C#3'],
    'b-minor' : ['B3', 'F#3', 'D3'],
    'c-minor' : ['C3', 'G3', 'E-3'],
    'c#-minor' : ['C#3', 'G#3', 'E3'],
    'd-minor' : ['D3', 'A3', 'F3'],
    'e--minor' : ['E-3', 'B-3', 'F#3']

}

In [108]:
chord_notes = {
    'c-major' : ['C4', 'E4', 'G4'],
    'c#-major' : ['C#4', 'F4', 'G#4'],
    'd-major' : ['D4', 'A4', 'F#4'],
    'e--major': ['E-4', 'B-4', 'G4'],
    'e-major' : ['E4', 'B4', 'G#4'],
    'f-major' : ['F4', 'C4', 'A4'],
    'f#-major' : ['F#4', 'C#4,' 'B-4'],
    'g-major' : ['G4', 'B4', 'D4'],
    'g#-major' : ['G#4', 'C4', 'E-4'],
    'a-major' : ['A4', 'E4', 'C#4'],
    'b--major': ['B-4', 'F4', 'D4'],
    'b-major' : ['B4', 'F#4', 'E-4'],
    
    'e-minor' : ['E4', 'B4', 'G4'],
    'f-minor' : ['F4', 'C4', 'G#4'],
    'f#-minor' : ['F#4', 'C#4', 'A4'],
    'g-minor' : ['G4', 'D4', 'B-4'],
    'g#-minor' : ['G#4', 'E-4', 'B4'],
    'a-minor' : ['A4', 'E4', 'C4'],
    'b--minor' : ['B-4', 'F4', 'C#4'],
    'b-minor' : ['B4', 'F#4', 'D4'],
    'c-minor' : ['C4', 'G4', 'E-4'],
    'c#-minor' : ['C#4', 'G#4', 'E4'],
    'd-minor' : ['D4', 'A4', 'F4'],
    'e--minor' : ['E-4', 'B-4', 'F#4']

}

In [109]:
def three_in_a_row(new_note, one_hot_new_data, i):
    entry = encoder.inverse_transform(new_note)
    last = encoder.inverse_transform(one_hot_new_data[i-1:i])
    before_last = encoder.inverse_transform(one_hot_new_data[i-2:i-1])
    if entry != last or entry != before_last:
        return False
    return True

In [110]:
def choose_another_note(forbidden, curr_chord):
    our_scale = set(scales[input_scale])
    curr_chord_notes = set(chord_notes[curr_chord])
    if np.any(forbidden):
        forbidden = np.reshape(forbidden, (1, -1))
        forbidden = encoder.inverse_transform(forbidden)
        our_scale_valid = our_scale.difference({forbidden[0][0]})
        curr_chord_notes_valid = curr_chord_notes.difference({forbidden[0][0]})
    else:
        our_scale_valid = our_scale
        curr_chord_notes_valid = curr_chord_notes
        
    rand_list = []
    if random.random() < 0.7:
        rand_list = [random.choice(list(curr_chord_notes_valid))]
    else:
        rand_list = [random.choice(list(our_scale_valid))]
    new_note = np.array(rand_list)
    new_note = np.reshape(new_note, (len(new_note), 1))

    new_note = encoder.transform(new_note)
    new_note = new_note.toarray()
    
    return new_note
    
    

In [112]:
our_scale = set(scales[input_scale])

def in_scale(new_note):
    if encoder.inverse_transform(new_note)[0][0] not in our_scale:
        return False
    return True

In [114]:
progression = {
    'c-major' : ['a-minor', 'c-major', 'f-major', 'g-major'],
    'a-minor' : ['a-minor', 'c-major', 'f-major', 'g-major'],
    
    'c#-major' : ['b--minor', 'c#-major', 'f#-major', 'g#-major'],
    'b--minor' : ['b--minor', 'c#-major', 'f#-major', 'g#-major'],
    
    'd-major' : ['b-minor', 'd-major', 'g-major', 'a-major'],
    'b-minor' : ['b-minor', 'd-major', 'g-major', 'a-major'],
    
    'e--major' : ['c-minor', 'e--major', 'g#-major', 'b--major'],
    'c-minor' : ['c-minor', 'e--major', 'g#-major', 'b--major'],
    
    'e-major' : ['c#-minor', 'e-major', 'a-major', 'b-major'],
    'c#-minor' : ['c#-minor', 'e-major', 'a-major', 'b-major'],
    
    'f-major' : ['d-minor', 'f-major', 'b--major', 'c-major'],
    'd-minor' : ['d-minor', 'f-major', 'b--major', 'c-major'],
    
    'f#-major' : ['e--minor', 'f#-major', 'b-major', 'c#-major'],
    'e--minor' : ['e--minor', 'f#-major', 'b-major', 'c#-major'],
    
    'g-major' : ['e-minor', 'g-major', 'c-major', 'd-major'],
    'e-minor' : ['e-minor', 'g-major', 'c-major', 'd-major'],
    
    'g#-major' : ['f-minor', 'g#-major', 'c#-major', 'e--major'],
    'f-minor' : ['f-minor', 'g#-major', 'c#-major', 'e--major'],
    
    'a-major' : ['f#-minor', 'a-major', 'd-major', 'e-major'],
    'f#-minor' : ['f#-minor', 'a-major', 'd-major', 'e-major'],
    
    'b--major' : ['g-minor', 'b--major', 'e--major', 'f-major'],
    'g-minor' : ['g-minor', 'b--major', 'e--major', 'f-major'],
    
    'b-major' : ['g#-minor', 'b-major', 'e-major', 'f#-major'],
    'g#-minor' : ['g#-minor', 'b-major', 'e-major', 'f#-major'],
    
    
    
}

In [116]:
song_notes_len = 256
accompaniment = stream.Part()

chords_acc = {0 : progression[input_scale][0], 1 : progression[input_scale][1],
              2 : progression[input_scale][2], 3 : progression[input_scale][3]}

cicles = int(song_notes_len/4)

for j in range(cicles):
    if j % 4 == 0:
        for k in range(4):
            e_min_chord = chord.Chord(acc_chords[chords_acc[0]])
            e_min_chord.duration.type = 'quarter'
            accompaniment.append(e_min_chord)
    if j % 4 == 1:
        for k in range(4):
            g_maj_chord = chord.Chord(acc_chords[chords_acc[1]])
            g_maj_chord.duration.type = 'quarter'
            accompaniment.append(g_maj_chord)
    if j % 4 == 2:
        for k in range(4):
            c_maj_chord = chord.Chord(acc_chords[chords_acc[2]])
            c_maj_chord.duration.type = 'quarter'
            accompaniment.append(c_maj_chord)
    if j % 4 == 3:
        for k in range(4):
            d_maj_chord = chord.Chord(acc_chords[chords_acc[3]])
            d_maj_chord.duration.type = 'quarter'
            accompaniment.append(d_maj_chord)


In [118]:
def walkOption(lastNote, fromNote, toNotes, curr_chord, measurePosition):
    lastNote = note.Note(lastNote)
    fromNote = note.Note(fromNote)
    note_from_chord = note.Note(toNotes[0])
    if measurePosition == 3:
        if lastNote.nameWithOctave not in curr_chord:
            return ""
        intervalFirst = interval.notesToInterval(lastNote, fromNote)
        intervalSecond = interval.notesToInterval(fromNote, note_from_chord)
        if (intervalFirst.semitones == 2 or intervalFirst.semitones ==1) and intervalSecond.semitones == 1:
            return note_from_chord.nameWithOctave
        if (intervalFirst.semitones == -2 or intervalFirst.semitones == -1) and intervalSecond.semitones == -1:
            return note_from_chord.nameWithOctave
        
    elif measurePosition == 2:
        if fromNote.nameWithOctave in our_scale:
            intervalThird = interval.notesToInterval(fromNote, note_from_chord)
            if intervalThird.semitones == 3:
                i = interval.Interval(2)
                add_note = fromNote.transpose(i)
                if add_note.nameWithOctave in our_scale:
                    return add_note.nameWithOctave
            elif intervalThird.semitones == -3:
                i = interval.Interval(-2)
                add_note = fromNote.transpose(i)
                if add_note.nameWithOctave in our_scale:
                    return add_note.nameWithOctave
        
        if lastNote.nameWithOctave not in curr_chord:
            return ""
        intervalFourth = interval.notesToInterval(fromNote, note_from_chord)
        if intervalFourth.semitones == 2:
            i = interval.Interval(1)
            add_note = fromNote.transpose(i)
            return add_note.nameWithOctave
        if intervalFourth.semitones == -2:
            i = interval.Interval(-1)
            add_note = fromNote.transpose(i)
            return add_note.nameWithOctave
    return ""

In [119]:
i = input_len
curr_chord = 0
last_note = one_hot_new_data[input_len-1]
last_note = np.reshape(last_note, (1, one_hot_data.shape[1]))

cancel_counter = 0
walk_counter = 0

while i < song_notes_len:
    curr_chord = chords_acc[int(i / 4.0) % 4]
    next_chord = chords_acc[int((i+2) / 4.0) % 4]
    new_note = model.predict(np.reshape(one_hot_new_data[i-input_len:i], (1, input_len, one_hot_data.shape[1])))
    new_note = np.array(list(map(funkcija, new_note[0])))
    new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))

    while not np.any(new_note[0]):
        new_note = choose_another_note(new_note[0], curr_chord)
        new_note = np.array(list(map(funkcija, new_note[0])))
        new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
        
    add_note = walkOption(encoder.inverse_transform(last_note)[0][0], encoder.inverse_transform(new_note)[0][0], chord_notes[next_chord], chord_notes[curr_chord], i%4)
    if add_note != "":
        walk_counter += 1
        #print(encoder.inverse_transform(new_note)[0][0])
        #print(add_note)

        add_note_np = np.array([add_note])
        add_note_np = np.reshape(add_note_np, (len(add_note_np), 1))
        one_hot_add_note = encoder.transform(add_note_np)
        one_hot_add_note = one_hot_add_note.toarray()
        
        one_hot_new_data = np.append(one_hot_new_data, new_note, axis=0)
        one_hot_new_data = np.append(one_hot_new_data, one_hot_add_note, axis=0)
                
        if i % 4 == 2:
            new_note_str = encoder.inverse_transform(new_note)[0][0]
            interval_my = interval.notesToInterval(note.Note(new_note_str), note.Note(add_note))

            to_note = note.Note(add_note).transpose(interval.Interval(interval_my.semitones)).nameWithOctave
            
            if to_note not in our_scale:
                to_note = note.Note(to_note).transpose(interval.Interval(-1)).nameWithOctave
            #print(to_note)
            
            to_note = np.array([to_note])
            to_note = np.reshape(to_note, (len(to_note), 1))
            one_hot_to_note = encoder.transform(to_note)
            one_hot_to_note = one_hot_to_note.toarray()
            one_hot_new_data = np.append(one_hot_new_data, one_hot_to_note, axis=0)
            i+=1
        i += 2
    
            
        continue
    # not in_scale(new_note) 
    while not in_scale(new_note) or three_in_a_row(new_note, one_hot_new_data, i):
        cancel_counter+=1
        new_note = choose_another_note(new_note[0], curr_chord)
        new_note = np.array(list(map(funkcija, new_note[0])))
        new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
        
    one_hot_new_data = np.append(one_hot_new_data, new_note, axis=0)
    last_note = new_note
    i += 1



1/1 [==============================] - 0s 63ms/step
D4
C#4
C4
1/1 [==============================] - 0s 68ms/step
E4
E-4
D4
1/1 [==============================] - 0s 64ms/step
E4
F#4
G4
1/1 [==============================] - 0s 57ms/step
E4
F#4
G4
1/1 [==============================] - 0s 65ms/step
E4
F#4
G4
1/1 [==============================] - 0s 47ms/step
E4
F#4
G4
1/1 [==============================] - 0s 64ms/step
E4
E-4
D4
1/1 [==============================] - 0s 45ms/step
E4
E-4
D4
1/1 [==============================] - 0s 50ms/step
E4
F#4
G4
1/1 [==============================] - 0s 45ms/step
E4
E-4
D4
1/1 [==============================] - 0s 56ms/step
E4
E-4
D4
1/1 [==============================] - 0s 45ms/step
A4
G#4
G4
1/1 [==============================] - 0s 45ms/step


In [120]:
one_hot_new_data = encoder.inverse_transform(one_hot_new_data)

In [121]:
one_hot_new_data

array([['E4'],
       ['F#4'],
       ['G4'],
       ['F#4'],
       ['D4'],
       ['E4'],
       ['G4'],
       ['A4'],
       ['G4'],
       ['B4'],
       ['A4'],
       ['G4'],
       ['F#4'],
       ['E4'],
       ['D4'],
       ['E4'],
       ['E4'],
       ['G4'],
       ['G4'],
       ['B4'],
       ['A4'],
       ['A4'],
       ['G4'],
       ['A4'],
       ['A4'],
       ['G4'],
       ['A4'],
       ['B4'],
       ['A4'],
       ['B4'],
       ['A4'],
       ['B4'],
       ['B4'],
       ['G4'],
       ['B4'],
       ['A4'],
       ['A4'],
       ['D4'],
       ['D4'],
       ['C#4'],
       ['C4'],
       ['D4'],
       ['E4'],
       ['B4'],
       ['G4'],
       ['A4'],
       ['A4'],
       ['G4'],
       ['A4'],
       ['B4'],
       ['B4'],
       ['E4'],
       ['A4'],
       ['B4'],
       ['A4'],
       ['G4'],
       ['G4'],
       ['A4'],
       ['B4'],
       ['G4'],
       ['E4'],
       ['E4'],
       ['F#4'],
       ['E4'],
       ['D4'],
       ['B4'],
     

In [123]:
def notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        note_snip = note.Note(i)
        note_snip.offset = offset
        Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

In [126]:
Melody = notes(one_hot_new_data.reshape(one_hot_new_data.shape[0], ))
score = stream.Score()
score.insert(0, Melody)
score.insert(0, accompaniment)
flat_stream = accompaniment.flat
counter = 1
# iterate over the elements of the flattened stream and print their types
for element in flat_stream:
    if element.isNote:
        print('Note:', element.pitch)
    elif element.isChord:
        counter+=1
        print('Chord:', element.pitches)
score.write('midi', 'something.mid')